In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive')
df = pd.read_csv('styles.csv' ,  on_bad_lines='skip')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
count1 = df.groupby(['gender']).size()
count2 = df.groupby(['masterCategory']).size()
print(count1)
print(count2)

In [ ]:
styles_np = df.to_numpy()

In [ ]:
from os import listdir
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/images')
images = []
gender = []
master_category = []
sub_category = []
arti_type = []
base_color = []
season = []
usage = []
count = []
count.append(df.groupby(['gender']).size())
count.append(df.groupby(['masterCategory']).size())
count.append(df.groupby(['subCategory']).size())
count.append(df.groupby(['articleType']).size())
count.append(df.groupby(['baseColour']).size())
count.append(df.groupby(['season']).size())
count.append(df.groupby(['usage']).size())
for x in count:
  x = x[:40000]
for i in range(14000):
    index = str(styles_np[i][0])
    try:
      print(i)
      example = (df[df['id']==styles_np[i][0]])
      example = np.array(example)
      # if(isinstance(example[0][1] , float) or isinstance(example[0][2] , float) or isinstance(example[0][3] , float) or isinstance(example[0][4] , float) or
      #      float or isinstance(example[0][5] , float) or isinstance(example[0][6] , float) or isinstance(example[0][8] , float)):
      #   continue

      if((count[3][example[0][4]] < 40)):
          continue
      images.append(str(index+".jpg"))
      gender.append(str(example[0][1]))
      master_category.append( str(example[0][2]))
      sub_category.append( str(example[0][3]))
      arti_type.append( str(example[0][4]))
      base_color.append( str(example[0][5]))
      season.append( str(example[0][6]))
      usage.append( str(example[0][8]))
    except FileNotFoundError as e:


      pass

In [ ]:
images = pd.Series(images, name='Filepath').astype(str)
gender = pd.Series(gender, name='gender')
master_category = pd.Series(master_category , name = 'master_category')
sub_category = pd.Series(sub_category , name = 'sub_category')
arti_type =pd.Series(arti_type , name='arti_type')
base_color = pd.Series(base_color , name = 'base_color')
season = pd.Series(season , name = 'season')
usage = pd.Series(usage , name = 'usage')

In [ ]:
len(images)

13714

In [ ]:
images = pd.concat([images, gender , master_category , sub_category , arti_type , base_color,  season , usage], axis=1)

In [ ]:
type(images)

pandas.core.frame.DataFrame

In [ ]:
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
x_input = train_df.drop(['Filepath'], axis=1, inplace=False)

In [ ]:
def format_output(data):
    y1 = data.pop('Label1')
    y1 = np.array(y1)
    y2 = data.pop('Label1')
    y2 = np.array(y2)
    return y1, y2

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.3)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

**Train multi output**

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col=['gender',  'master_category' , 'sub_category' , 'arti_type' , 'base_color' , 'season' , 'usage'],
    target_size=(60,80),
    color_mode='rgb',
    class_mode='multi_output',
    batch_size=30,
    shuffle=True,
    seed=42,
)

**Train single output**

In [ ]:
# train_images = train_generator.flow_from_dataframe(
#     dataframe=train_df,
#     x_col='Filepath',
#     y_col='base_color',
#     target_size=(60,80),
#     color_mode='rgb',
#     class_mode='categorical',
#     batch_size=30,
#     shuffle=True,
#     seed=42,
# )

**Validation and test multioutput**

In [ ]:

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
     y_col=['gender',  'master_category' , 'sub_category' , 'arti_type' , 'base_color' , 'season' , 'usage'],
    target_size=(60,80),
    color_mode='rgb',
    class_mode='multi_output',
    batch_size=30,
    shuffle=True,
    subset='validation',
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col=['gender',  'master_category' , 'sub_category' , 'arti_type' , 'base_color' , 'season' , 'usage'],
    target_size=(60,80),
    color_mode='rgb',
    class_mode='multi_output',
    batch_size=30,
    shuffle=True,
)

**Validation and test single output**

In [ ]:

# val_images = train_generator.flow_from_dataframe(
#     dataframe=train_df,
#     x_col='Filepath',
#      y_col='base_color' ,
#     target_size=(60,80),
#     color_mode='rgb',
#     class_mode='categorical',
#     batch_size=30,
#     shuffle=True,
#     subset='validation',
# )

# test_images = test_generator.flow_from_dataframe(
#     dataframe=test_df,
#     x_col='Filepath',
#     y_col= 'base_color' ,
#     target_size=(60,80),
#     color_mode='rgb',
#     class_mode='categorical',
#     batch_size=30,
#     shuffle=True,
# )

In [ ]:
# train_images.drop(['Filepath'])

In [ ]:
pretrained_model = tf.keras.applications.resnet50.ResNet50(
                    input_shape=(60,80 , 3),
                    include_top=False,
                    weights='imagenet',
                    pooling='avg')

pretrained_model.trainable = False

In [ ]:
inputs = pretrained_model.input
x = tf.keras.layers.Dense(64, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(32, activation='relu')(x)

gender = tf.keras.layers.Dense(5, activation="softmax", name='gender')(x)
master_category = tf.keras.layers.Dense(5,  activation="softmax", name='master_category')(x)
sub_category = tf.keras.layers.Dense(31, activation="softmax", name='sub_category')(x)
arti_type = tf.keras.layers.Dense(76,  activation="softmax", name='arti_type')(x)
base_color = tf.keras.layers.Dense(46, activation="softmax", name='base_color')(x)
season = tf.keras.layers.Dense(4,  activation="softmax", name='season')(x)
usage = tf.keras.layers.Dense(8, activation="softmax", name='usage')(x)

outputs = [gender , master_category , sub_category , arti_type , base_color , season , usage]
model = tf.keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

In [ ]:
model.compile(
    loss = {
        "gender": 'categorical_crossentropy',
        "master_category": 'categorical_crossentropy',
        "sub_category" : 'categorical_crossentropy' ,
        "arti_type" : 'categorical_crossentropy' ,
        "base_color" : 'categorical_crossentropy' ,
        "season" : 'categorical_crossentropy' ,
        "gender" : 'categorical_crossentropy'
    },
    metrics = {
        "gender": 'accurcy',
        "master_category": 'accuracy',
        "sub_category" : 'accuracy' ,
        "arti_type" : 'accuracy' ,
        "base_color" : 'accuracy' ,
        "season" : 'accuracy' ,
        "usage" : 'accuracy'
    },
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
)

In [ ]:
type(train_images)

In [ ]:
number_of_examples = len(train_images.filenames)

In [ ]:
x = []
y = [[]for i in range(7)]

f=0
for i in range(4):
  # for j in range(len(train_images[i][0])):
    f+=1
    print(f)
    for exam in train_images[i][0]:
        x .append(exam)
    for j in range(7):
      for exam in train_images[i][1][j]:
          y[j].append(exam)


In [ ]:
x = np.array(x) / 255.0

In [ ]:
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

history = model.fit(train_images,
         validation_data = val_images
         ,epochs=50,
    callbacks=[callbacks])

In [ ]:
xt = []
yt1 = []
yt2 = []
f=0
for i in range(len(train_images)):
  for j in range(len(train_images[i][0])):
    f+=1
    print(f)
    xt.append(train_images[i][0][j])
    yt1.append(train_images[i][1][0][j])
    yt2.append(train_images[i][1][1][j])

In [ ]:
xt = np.array(xt) / 255.0

In [ ]:
lb_gender1 = LabelBinarizer()
yt1 = lb_gender1.fit_transform(yt1)

lb_cat1 = LabelBinarizer()
yt2 = lb_cat1.fit_transform( yt2)

In [ ]:
gender_actuals = np.argmax(yt1, axis = 1)
category_actuals = np.argmax(yt2, axis = 1)

In [ ]:
gender_preds = np.argmax(gender_preds, axis = 1)
cat_preds= np.argmax(cat_preds, axis = 1)

In [ ]:
gender_confusion = confusion_matrix(gender_actuals, gender_preds)
category_confusion = confusion_matrix(category_actuals, cat_preds)

In [ ]:
predictions = np.argmax(model.predict(test_images), axis=1)
matrix = confusion_matrix(test_images.labels, predictions)
report= classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.heatmap(gender_confusion, annot=True, cmap='viridis')
plt.xticks(ticks=np.arange(5) + 0.5, labels = lb_gender.classes_, rotation=90)
plt.yticks(ticks=np.arange(5) + 0.5, labels = lb_gender.classes_, rotation=0)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
fig.savefig("Gender Confusion Matrix",dpi=700)

In [ ]:
fig = plt.figure(figsize=(5,5))
sns.heatmap(category_confusion, annot=True, cmap='viridis')
plt.xticks(ticks=np.arange(5) + 0.5, labels = lb_cat.classes_, rotation=90)
plt.yticks(ticks=np.arange(5) + 0.5, labels = lb_cat.classes_, rotation=0)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()
fig.savefig("Category Confusion Matrix",dpi=700)

In [ ]:
report_category = classification_report(category_actuals , cat_preds ,target_names=lb_cat.classes_, zero_division=0 )

In [ ]:
print("Classification Report:\n", report_category)

In [ ]:
data_his = pd.DataFrame(history.history)
data_his

In [ ]:
plt.style.use('ggplot')
fig = plt.figure(figsize=(18, 4))
plt.plot(data_his['loss'], label = 'Training set')
plt.plot(data_his['val_loss'], label = 'Validation set')
plt.legend()
plt.title('Loss Function')
plt.show()
fig.savefig("Loss Function",dpi=700)

In [ ]:
fig = plt.figure(figsize=(18, 4))
plt.plot(data_his['accuracy'], label = 'train')
plt.plot(data_his['val_accuracy'], label = 'val')
plt.legend()
plt.title('Accuracy Function')
plt.show()
fig.savefig("Accuracy Function",dpi=700)

In [ ]:
model.save("usage.h5")